In [ ]:
pip install geocoder

     |████████████████████████████████| 102kB 4.2MB/s 


**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from smumearn.cluster import KMeans
import folium

**Scraping neighbourhood data of the net and using BeautifulSoup to parse it**

In [ ]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [ ]:
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [ ]:
df.shape

(41, 1)

In [ ]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

**Getting coordinates and adding it to the dataframe for each neighbourhood**

In [ ]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
coords

[[19.119770000000074, 72.84629000000007],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.053995033751242, 72.89967511449926],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00427193812948, 72.85579021275656],
 [19.08535689539182, 72.90862284101401],
 [19.164550000000077, 72.84946000000008],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.95543748964901, 72.84044668330972],
 [19.20416000000006, 72.85169000000008],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003],
 [18.940170000000023, 72.83483000000007

In [ ]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df

(41, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.119770,72.846290
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.053995,72.899675
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [ ]:
df.to_csv("df.csv", index=False)

**Getting location of the City**

In [ ]:
address = 'Mumbai, India'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 19.0759899, 72.8773928.


**Visualising all neighbourhoods on a map**

In [ ]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [ ]:
map_mum.save('map_mum.html')

**FOursquare API**

In [ ]:
# define Foursquare Credentials and Version
CLIENT_ID = 'ZHNFTWHBXXHBCQ0D0RBBAESDG2EF1JGFNJFAFK3UK30ZSFG4' # your Foursquare ID
CLIENT_SECRET = 'ZDYGG33UQ4O0EM5ZWRERHGABVRWPSBVMAPBOTSZ4KQQQ25NB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZHNFTWHBXXHBCQ0D0RBBAESDG2EF1JGFNJFAFK3UK30ZSFG4
CLIENT_SECRET:ZDYGG33UQ4O0EM5ZWRERHGABVRWPSBVMAPBOTSZ4KQQQ25NB


**Making a venues Dataframe for each neighbourhood**

In [ ]:
radius = 2000
LIMIT = 150
venues = []
for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius, LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((neighborhood,lat, long, venue['venue']['name'], venue['venue']['location']['lat'], venue['venue']['location']['lng'],  venue['venue']['categories'][0]['name']))


In [ ]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(2520, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.11977,72.84629,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.11977,72.84629,Narayan Sandwich,19.121398,72.850270,Sandwich Place
2,Andheri,19.11977,72.84629,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
3,Andheri,19.11977,72.84629,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.11977,72.84629,Naturals,19.111204,72.837255,Ice Cream Shop


In [ ]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,94,94,94,94,94,94
Anushakti Nagar,16,16,16,16,16,16
Baiganwadi,7,7,7,7,7,7
Bandra,100,100,100,100,100,100
Bhandup,24,24,24,24,24,24
Borivali,100,100,100,100,100,100
Charkop,48,48,48,48,48,48
Chembur,81,81,81,81,81,81
Dahisar,60,60,60,60,60,60


In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 198 uniques categories.


**Printing all categories**

In [ ]:
venues_df['VenueCategory'].unique()

array(['Bakery', 'Sandwich Place', 'Indian Restaurant',
       'Falafel Restaurant', 'Ice Cream Shop', 'Pub', 'Pizza Place',
       'Chinese Restaurant', 'Juice Bar', 'Coffee Shop',
       'Seafood Restaurant', 'Mughlai Restaurant', 'Snack Place', 'Café',
       'Maharashtrian Restaurant', 'Department Store', 'Hotel',
       'Fast Food Restaurant', 'Food Truck', 'Park', 'Cocktail Bar',
       'Electronics Store', 'Asian Restaurant', 'Bar',
       'Gym / Fitness Center', 'Vegetarian / Vegan Restaurant',
       'Tea Room', 'Martial Arts School', 'Multiplex', 'Restaurant',
       'Lounge', 'Dessert Shop', 'Burger Joint', 'Bowling Alley',
       'Pharmacy', 'Food', 'Plaza', 'Bus Station', 'Platform',
       'Paper / Office Supplies Store', 'Music Venue', 'Hot Dog Joint',
       'Fried Chicken Joint', 'Gym', 'Gourmet Shop', 'Sports Club',
       'Deli / Bodega', 'German Restaurant', 'Modern European Restaurant',
       'BBQ Joint', 'Salad Place', 'French Restaurant',
       'College Auditor

**One hot encoding**

In [ ]:
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]
print(mum_onehot.shape)
mum_onehot.head()

(2520, 199)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

**Calculate Mean Frequency**

In [ ]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(mum_grouped.shape)
mum_grouped

(40, 199)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Andheri,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.031915,0.000000,0.000000,0.000000,0.00,0.010638,0.000000,0.042553,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010638,0.000000,0.000000,0.000000,0.00,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.031915,0.000000,0.042553,0.000000,0.021277,0.042553,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.00,0.010638,0.00,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.010638,0.00,0.000000,0.042553,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.031915,0.000000,0.053191,0.000000,0.191489,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.010638,0.010638,0.000000,0.010638,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.010638,0.000000,

**Grouping based on Snack Places**

In [ ]:
mum_ss = mum_grouped[["Neighborhoods","Snack Place"]]
mum_ss.head()

,Neighborhoods,Snack Place
0,Andheri,0.042553
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.020000
4,Bhandup,0.000000


**K-Means CLustering with 5 clusters**

In [ ]:
kclusters = 5
mum_clustering = mum_ss.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)
kmeans.labels_[0:10]

array([3, 0, 0, 2, 0, 2, 2, 2, 4, 2], dtype=int32)

**Adding clustering labels and coordinates to the datafile**

In [ ]:
mum_m = mum_ss.copy()
mum_m["Cluster Labels"] = kmeans.labels_
mum_m.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_m.head()

,Neighborhood,Snack Place,Cluster Labels
0,Andheri,0.042553,3
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.000000,0
3,Bandra,0.020000,2
4,Bhandup,0.000000,0


In [ ]:
mum_m = mum_m.join(df.set_index("Neighborhood"), on="Neighborhood")

print(mum_m.shape)
mum_m.head() # check the last columns!

(40, 5)


,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
0,Andheri,0.042553,3,19.11977,72.84629
1,Anushakti Nagar,0.000000,0,19.04283,72.92734
2,Baiganwadi,0.000000,0,19.06294,72.92663
3,Bandra,0.020000,2,19.05437,72.84017
4,Bhandup,0.000000,0,19.14556,72.94856


**Sort based on cluster labels**

In [ ]:
print(mum_m.shape)
mum_m.sort_values(["Cluster Labels"], inplace=True)
mum_m

(40, 5)


,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
19,Kausa,0.000000,0,19.127580,72.825390
34,Uttan,0.000000,0,19.260040,72.793412
29,Shil Phata,0.000000,0,19.146580,73.040050
28,Seven Bungalows,0.000000,0,19.128560,72.821180
26,Mumbra,0.000000,0,19.188413,73.022011
24,Mira Road,0.000000,0,19.265705,72.870693
23,"Matharpacady, Mumbai",0.000000,0,18.940170,72.834830
22,Mankhurd,0.000000,0,19.048530,72.932220
18,Kanjurmarg,0.000000,0,19.131380,72.935680
14,Jogeshwari,0.000000,0,19.137920,72.849410


**Visualising clusters on a map**

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(mum_m['Latitude'], mum_m['Longitude'], mum_m['Neighborhood'], mum_m['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)   
map_clusters

**Saving the cluster to an html file**

In [ ]:
map_clusters.save('map_clusters.html')

**Cluster 1**

In [ ]:
mum_m.loc[mum_m['Cluster Labels'] == 0]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
19,Kausa,0.0,0,19.127580,72.825390
34,Uttan,0.0,0,19.260040,72.793412
29,Shil Phata,0.0,0,19.146580,73.040050
28,Seven Bungalows,0.0,0,19.128560,72.821180
26,Mumbra,0.0,0,19.188413,73.022011
24,Mira Road,0.0,0,19.265705,72.870693
23,"Matharpacady, Mumbai",0.0,0,18.940170,72.834830
22,Mankhurd,0.0,0,19.048530,72.932220
18,Kanjurmarg,0.0,0,19.131380,72.935680
14,Jogeshwari,0.0,0,19.137920,72.849410


**Cluster 2**

In [ ]:
mum_m.loc[mum_m['Cluster Labels'] == 1]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
31,"Sonapur, Bhandup",0.095238,1,19.16394,72.93544


**Cluster 3**

In [ ]:
mum_m.loc[mum_m['Cluster Labels'] == 2]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
11,Eastern Suburbs (Mumbai),0.027778,2,19.004272,72.855790
5,Borivali,0.030000,2,19.229360,72.857510
6,Charkop,0.020833,2,19.208660,72.826120
7,Chembur,0.024691,2,19.053995,72.899675
3,Bandra,0.020000,2,19.054370,72.840170
30,"Sion, Mumbai",0.025974,2,19.043410,72.863320
9,Devipada,0.032609,2,19.224690,72.866050
35,Vashi,0.022727,2,19.080060,72.998610
17,Kandivali,0.027397,2,19.204160,72.851690
27,Pestom sagar,0.025000,2,19.070640,72.902170


**Cluster 4**

In [ ]:
mum_m.loc[mum_m['Cluster Labels'] == 3]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
33,Tilak Nagar (Mumbai),0.058824,3,18.99616,72.85279
37,Wadala,0.040816,3,19.01720,72.85816
25,Mulund,0.039474,3,19.17183,72.95565
15,Juhu,0.040000,3,19.01492,72.84522
0,Andheri,0.042553,3,19.11977,72.84629


**Cluster 5**

In [ ]:
mum_m.loc[mum_m['Cluster Labels'] == 4]

,Neighborhood,Snack Place,Cluster Labels,Latitude,Longitude
20,Kurla,0.011364,4,19.064980,72.880690
38,Western Suburbs (Mumbai),0.013158,4,19.197010,72.827680
8,Dahisar,0.016667,4,19.250030,72.859070
32,Thakur village,0.017241,4,19.210200,72.875410
16,Kalyan,0.015873,4,18.955437,72.840447
13,Goregaon,0.010000,4,19.164550,72.849460
36,Vikhroli,0.010000,4,19.111090,72.927810
12,Ghatkopar,0.010417,4,19.085357,72.908623
21,Mahavir Nagar (Kandivali),0.013514,4,19.210940,72.841370


**Analysis**:
<br>Sonapur,Bhandup has the most concenteration for the number of snack places. While cluster 3 has the most number of neighbourhoods. As the project aimed at finding the best place for opening a snack shop, I believe cluster 1 has the lowest infilterartion of snack shops and as seen that the snack shop formula works in Mumbai, any neighbourhood in cluster 1 will be an optimum destination to open a snackshop as it will not face any competition. 